In [ ]:
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import pickle

with open('../paper_calculations/aligned_reconstruction_wo_tw_errors.pickle', 'rb') as handle:
      aligned_reconstruction_wo_tw_errors = pickle.load(handle)
        
keys = [a[1] for a in aligned_reconstruction_wo_tw_errors]
values = [a[2:] for a in aligned_reconstruction_wo_tw_errors]

aligned_notw_errors_dict = dict(zip(keys,values))

In [ ]:
#aligned_notw_errors_dict.keys()

In [ ]:
def clean(rawdata):
    return rawdata.replace("'","\"").replace("False","0").replace("True","1").replace("None","\"None\"")

In [ ]:
def safe_check_required_key_value(dictionary, key, value):
    return key in dictionary.keys() and dictionary[key] == value

In [ ]:
def match_dict_values(curdict, goaldict, ignore_keys_list):
    for key in goaldict.keys():
        if key not in ignore_keys_list:
            if not safe_check_required_key_value(curdict, key, goaldict[key]):
                return False
    return True

In [ ]:
def get_all_data():
    rawdata1 = pd.read_csv("../project.csv")
    rawdata1 = rawdata1.reset_index()  # make sure indexes pair with number of rows

    rawdata2 = pd.read_csv("../projectrateinvariantvae.csv")
    rawdata2 = rawdata2.reset_index()  # make sure indexes pair with number of rows


    rawdata = pd.concat([rawdata1,rawdata2])
    rawdata.reset_index()
    return rawdata

In [ ]:
rawdata = get_all_data()

all_run_dicts = []

for index, row in rawdata.iterrows():
    configdict = json.loads(clean(row.config))
    if not any([match_dict_values(configdict, prd,["logname","model_save_dir","latent_dim"]) for prd in all_run_dicts]):
        # "convolutional_decoder" is deprecated. use "convolutional_decoder_upsampling" results instead
        if configdict["decoder_name"] != "convolutional_decoder":
            all_run_dicts.append(configdict)

In [ ]:
all_run_dicts

In [ ]:
# This was used to debug and figure out that this run was aborted before it completed,
# so there was no saved model associated with it
#weirdrunname = '../results/rateinvariantvae/20231114-204924.596986/savedmodel'
#
#
#weird_run = [a for a in all_run_dicts if a["model_save_dir"] == weirdrunname]
#
#for index, row in rawdata.iterrows():
#    configdict = json.loads(clean(row.config))
#    rowdata = json.loads(clean(row.summary))
#    if configdict["model_save_dir"] != weirdrunname:
#        #print(weird_run)
#        #print(row.summary)

In [ ]:
def create_dataset_combine(sweep_dictionaries,ignore_keys_list = ["logname","model_save_dir","latent_dim"]):
    all_data = []
    for sweep_dictionary in sweep_dictionaries:
        data = create_dataset(sweep_dictionary,ignore_keys_list)
        all_data.append(data)
    all_data = np.concatenate(all_data, axis=0)
    return all_data

In [ ]:
# add a default maxstamp so results don't change if we add more results
def create_dataset(sweep_dictionary,ignore_keys_list = ["logname","model_save_dir","latent_dim"], maxstamp = None):
    data = []
    data_timestamps = []
    rawdata = get_all_data()

    for index, row in rawdata.iterrows():
        configdict = json.loads(clean(row.config))
        if match_dict_values(configdict,sweep_dictionary,ignore_keys_list):
            rowdata = json.loads(clean(row.summary))
            if "train_alignedRMSE" in rowdata.keys() and rowdata["_step"] >= 199:
                if maxstamp is None or configdict["logname"][-26:-4] <= maxstamp:
                    save_key = configdict["model_save_dir"]
                    data.append((configdict["latent_dim"],
                                 aligned_notw_errors_dict[save_key][0],#rowdata["train_alignedRMSE"],
                                 aligned_notw_errors_dict[save_key][1],#rowdata["test_alignedRMSE"],
                                 rowdata["train_noiselessRMSE"],
                                 rowdata["test_noiselessRMSE"],
                                 rowdata["train_KLD"]))
                    data_timestamps.append(configdict["logname"][-26:-4])
    data = np.array(data) 
    data_timestamps=np.array(data_timestamps)
    return data, data_timestamps

In [ ]:
datasets = [create_dataset(rd,maxstamp=None) for rd in all_run_dicts]

In [ ]:
def plot_with_average_variance_line(ax,data,color,label,markerstyle,stamps,allow_partial):
    #ax.scatter(data[:,0],data[:,1],c=color,alpha=1,marker=markerstyle)
    xvals = []
    yvals = []
    times = []
    stdyvals = []
    for x,y in data:
        if x not in xvals:
            matching_inds = np.where(data[:,0]==x)[0]
            corresponding_stamps = stamps[matching_inds]
            if (not allow_partial) and len(corresponding_stamps) < 5:
                continue
            xvals.append(x)
            most_recent_stamps = np.argsort(corresponding_stamps)[-5:]
            most_recent_matching_data = data[matching_inds[most_recent_stamps]]
            yvals.append(np.mean(most_recent_matching_data[:,1]))
            stdyvals.append(np.std(most_recent_matching_data[:,1]))
    xvals = np.array(xvals)
    yvals = np.array(yvals)
    stdyvals = np.array(stdyvals)
    order = np.argsort(xvals)
    if len(xvals) == 0:
        return
    ax.plot(xvals[order],yvals[order],c=color,label=label,zorder=2)
    ax.scatter(xvals[order],yvals[order],c=color,alpha=1,marker=markerstyle,s=100,edgecolors= "k",linewidth=1,zorder=3)
    ax.fill_between(xvals[order], yvals[order]-stdyvals[order], yvals[order]+stdyvals[order], facecolor=color, alpha=0.5,zorder=1)
    

In [ ]:
dmpresults = np.load("../paper_calculations/dmpmodels/dmp_results_simple.npy")
dmpresults = dmpresults[dmpresults[:,0]>0]
dmpresults = dmpresults[dmpresults[:,0]<17]
# repeat five times with dummy dates so that these are plotted correctly (not filtered out for too few runs)
datasets.append((dmpresults.repeat(5,0),np.ones(len(dmpresults)).repeat(5,0)))

In [ ]:
dmpresults

In [ ]:

pcaresults = np.load("../paper_calculations/pca_results.npy")
pcaresults = pcaresults[pcaresults[:,0]>0]
pcaresults = pcaresults[pcaresults[:,0]<17]
# repeat five times with dummy dates so that these are plotted correctly (not filtered out for too few runs)
datasets.append((pcaresults.repeat(5,0),np.ones(len(pcaresults)).repeat(5,0)))

In [ ]:
pcaresults

In [ ]:
datasets[-1]

In [ ]:
import matplotlib.lines as mlines 
#https://stackoverflow.com/questions/47391702/how-to-make-a-colored-markers-legend-from-scratch

In [ ]:
import pickle


In [ ]:
def save_plots(namebase,rateylim,toplot = None,labels=None,colors=None,shapes=None,labeltextcolor="white",allow_partial=False):
    
    fsize=(8,6)
    odds = [i for i in range(1,17) if i % 5 == 1]
    
    
    if toplot is None:
        toplot = range(len(datasets))
    if labels is None:
        labels = toplot
    if colors is None:
        colors = [f"C{i}" for i in range(len(labels))]
    if shapes is None:
        shapes = ["o"] * len(datasets)
        
    print((toplot,labels,colors,shapes))
        
    f = plt.figure(figsize=fsize)
    ax = f.add_axes([0.1, 0.1, 0.8, 0.8])  # add the left Axes
    for i,label,color,shape in zip(toplot,labels,colors,shapes):
        dataval, stamps = datasets[i]
        if len(dataval.shape) > 1 and dataval.shape[1] > 5: # No KL for PCA
            plotdat = np.copy(dataval[:,(0,5)])
            plotdat[:,1] = np.log2(plotdat[:,1])
            plot_with_average_variance_line(ax, plotdat, color,label=f"{label}",markerstyle=shape, stamps=stamps,allow_partial=allow_partial)
    ax.set_ylim(2,np.log2(rateylim))
    ax.set_xlim(0.5,16.5)
    ax.set_xticks(odds,odds)
    ax.set_xticks(range(1,17),minor=True)
    plt.savefig(f"{namebase}LatentSweepRate.pdf",bbox_inches="tight")
    
    f = plt.figure(figsize=fsize)
    ax = f.add_axes([0.1, 0.1, 0.8, 0.8])  # add the left Axes
    for i,label,color,shape in zip(toplot,labels,colors,shapes):
        dataval, stamps = datasets[i]
        if len(dataval.shape) > 1:
            plot_with_average_variance_line(ax, dataval[:,(0,1)],color,label=f"{label}",markerstyle=shape, stamps=stamps,allow_partial=allow_partial)
    ax.set_ylim(0,1.2)
    ax.set_xlim(0.5,16.5)
    ax.set_xticks(odds,odds)
    ax.set_xticks(range(1,17),minor=True)
    plt.savefig(f"{namebase}LatentSweepTrain.pdf",bbox_inches="tight")
    
    f = plt.figure(figsize=fsize)
    ax = f.add_axes([0.1, 0.1, 0.8, 0.8])  # add the left Axes
    legend_lines = []
    for i,label,color,shape in zip(toplot,labels,colors,shapes):
        dataval, stamps = datasets[i]
        if len(dataval.shape) > 1:
            plot_line_info = plot_with_average_variance_line(ax, dataval[:,(0,2)],color,label=f"{label}",markerstyle=shape, stamps=stamps,allow_partial=allow_partial)
            legend_lines.append(
                mlines.Line2D([], [], color=color, marker=shape, label=f"{label}",markersize=10,markeredgecolor= "k",markeredgewidth=1))
            
    ax.set_ylim(0,1.2)
    ax.set_xlim(0.5,16.5)
    ax.set_xticks(odds,odds)
    ax.set_xticks(range(1,17),minor=True)
    plt.legend(handles=legend_lines,labelcolor=labeltextcolor, frameon=False,labelspacing=0.4)
    plt.savefig(f"{namebase}LatentSweepTest.pdf",bbox_inches="tight")
    plt.show()

In [ ]:
def is_base_config(testdict,must_not_match=[]):
    checks = [("beta",0.001),
                ("scalar_timewarper_name","modeled_scalar_timewarper"),
                ("decoder_name","functional_decoder_complicated"),
                ("dec_complicated_function_hidden_dims",[200]),
                ("training_data_added_timing_noise",0.1),
                ("vector_timewarper_name","identity_vector_timewarper"),
                ("encoder_name", "convolutional_encoder"),
                ("use_rate_invariant_autoencoder",1),
                ("use_rate_invariant_vae",1),
                ("device","cuda"),
                ("emb_nonlinearity","ReLU"),
                ("dec_use_elu",True),
                ("dec_use_tanh",True)
             ]
    for check in checks:
        #print(check[0])
        if check[0] in must_not_match:
            if safe_check_required_key_value(testdict,check[0],check[1]):
                #print("The problem was NOTMATCH",testdict,check[0],check[1])
                return False
        elif not safe_check_required_key_value(testdict,check[0],check[1]):
            #print("The problem was",testdict,check[0],check[1])
            return False
    return True

In [ ]:
[d for d in all_run_dicts if safe_check_required_key_value(d,"use_rate_invariant_autoencoder",False)]

In [ ]:
all_run_dicts[1]

In [ ]:
[d for d in all_run_dicts if is_base_config(d,["use_rate_invariant_autoencoder","use_rate_invariant_vae","dec_use_tanh"])]

In [ ]:
[d for d in all_run_dicts if is_base_config(d,["beta","decoder_name","dec_complicated_function_hidden_dims","scalar_timewarper_name","use_rate_invariant_vae","dec_use_tanh"])]

In [ ]:
#doesn't have a flag for dec_use_tanh, so we mark it here
base_config_index = np.where(np.array([is_base_config(d,("use_rate_invariant_autoencoder","use_rate_invariant_vae","dec_use_tanh")) for d in all_run_dicts]))[0].item()
noise_off_index = np.where(np.array([is_base_config(d,["training_data_added_timing_noise","use_rate_invariant_autoencoder","use_rate_invariant_vae","dec_use_tanh"]) for d in all_run_dicts]))[0].item()
timewarp_PCA_index = np.where(np.array([is_base_config(d,["dec_complicated_function_hidden_dims","use_rate_invariant_autoencoder","use_rate_invariant_vae","dec_use_tanh"]) for d in all_run_dicts]))[0].item()
notimewarp_index = np.where(np.array([is_base_config(d,["scalar_timewarper_name","use_rate_invariant_autoencoder","use_rate_invariant_vae","dec_use_tanh"]) for d in all_run_dicts]))[0].item()

conv_index = np.where(np.array([is_base_config(d,["decoder_name","dec_complicated_function_hidden_dims","scalar_timewarper_name","use_rate_invariant_autoencoder","use_rate_invariant_vae","dec_use_tanh"]) for d in all_run_dicts]))[0].item()
no_noise_timewarp_PCA_index = np.where(np.array([is_base_config(d,["training_data_added_timing_noise","dec_complicated_function_hidden_dims","use_rate_invariant_autoencoder","use_rate_invariant_vae","dec_use_tanh"]) for d in all_run_dicts]))[0].item()
no_timewarp_PCA_index = np.where(np.array([is_base_config(d,["scalar_timewarper_name","dec_complicated_function_hidden_dims","use_rate_invariant_autoencoder","use_rate_invariant_vae","dec_use_tanh"]) for d in all_run_dicts]))[0].item()

beta_inds = list(np.where(np.array([is_base_config(d,["beta","use_rate_invariant_autoencoder","use_rate_invariant_vae","dec_use_tanh"]) for d in all_run_dicts]))[0])
no_tw_beta_inds = list(np.where(np.array([is_base_config(d,["beta","scalar_timewarper_name","use_rate_invariant_autoencoder","use_rate_invariant_vae","dec_use_tanh"]) for d in all_run_dicts]))[0])
conv_beta_inds = list(np.where(np.array([is_base_config(d,["beta","decoder_name","dec_complicated_function_hidden_dims","scalar_timewarper_name","use_rate_invariant_autoencoder","use_rate_invariant_vae","dec_use_tanh"]) for d in all_run_dicts]))[0])

dtw_index = np.where(np.array([is_base_config(d,["vector_timewarper_name", "scalar_timewarper_name","use_rate_invariant_autoencoder","use_rate_invariant_vae","dec_use_tanh"]) for d in all_run_dicts]))[0].item()
#trans_index = np.where(np.array([is_base_config(d,["encoder_name","scalar_timewarper_name"]) for d in all_run_dicts]))[0].item()

ria_index = np.where(np.array([is_base_config(d,["scalar_timewarper_name","decoder_name","dec_complicated_function_hidden_dims","use_rate_invariant_autoencoder","emb_nonlinearity","dec_use_elu"]) for d in all_run_dicts]))[0].item()

# 4 + 3 + 2 + 2 + 2
print(len(all_run_dicts))

In [ ]:
[all_run_dicts[b] for b in beta_inds]

In [ ]:
print(all_run_dicts[ria_index])

In [ ]:
def print_matching_runs(sweep_dictionary, latent_dim=16, ignore_keys_list = ["logname","model_save_dir","latent_dim"], minstamp = None):
    data = []
    data_timestamps = []
    rawdata = get_all_data()

    for index, row in rawdata.iterrows():
        configdict = json.loads(clean(row.config))
        if (match_dict_values(configdict,sweep_dictionary,ignore_keys_list)
                             and configdict["latent_dim"]==latent_dim):
            print(configdict["logname"][:-4])
            


In [ ]:
for name, idval in [("timewarp:", base_config_index),
                    ("nodataaug:", noise_off_index),
                    ("nononlin:", timewarp_PCA_index),
                    ("noTimewarp:", notimewarp_index),
                    ("TimewarpVAEDTW", dtw_index),
                    ("conv",conv_index),
                    ("ria",ria_index)
                   ]:
    print(name)                
    print_matching_runs(all_run_dicts[idval])

In [ ]:
import matplotlib
font = {        'size'   : 22}

matplotlib.rc('font', **font)

In [ ]:
save_plots("ablation",rateylim=115,toplot=[base_config_index,noise_off_index,timewarp_PCA_index,notimewarp_index],
           labels=["TimewarpVAE","ndaug","nnonlin","NoTimewarp"],
          shapes=["o","P","X","."])

In [ ]:
save_plots("conv",rateylim=500,toplot=[base_config_index,no_timewarp_PCA_index,no_noise_timewarp_PCA_index],
           labels=["TimewarpVAE","NoTWNoNonlinear","NoNoiseNoNonlinear"])

In [ ]:
save_plots("beta",rateylim=100,toplot=[base_config_index] + beta_inds)
save_plots("BetaNoTw",rateylim=100,toplot=[base_config_index] + no_tw_beta_inds + [notimewarp_index])

In [ ]:
def rate_distortion(namebase,rateylim,latent_dim,toplot,colors=None,shapes=None):
    fsize=(8,6)
    if colors is None:
        colors = [f"C{i}" for i in range(2)]
    if shapes is None:
        shapes = ["o"] * len(datasets)
        
    f = plt.figure(figsize=fsize)
    ax = f.add_axes([0.1, 0.1, 0.8, 0.8])  # add the left Axes
    for i in toplot:
        dataval = datasets[i][0]
        dataval = dataval[dataval[:,0]==latent_dim]
        ax.scatter(dataval[:,5],dataval[:,1],c=colors[0],label=f"Training" if i == 0 else None,marker=shapes[0])
        ax.scatter(dataval[:,5],dataval[:,2],c=colors[1],label=f"Test" if i == 0 else None,marker=shapes[1])
    ax.set_xlim(0,rateylim)
    ax.set_ylim(0,1)
    plt.legend()
    plt.savefig(f"{namebase}RateDistortion.pdf",bbox_inches="tight")
    

In [ ]:
rate_distortion("base",100,16,[base_config_index] + beta_inds,colors=None,shapes=None)

In [ ]:
# These base distortion results make sense compared to bottom right of figure 3 in http://proceedings.mlr.press/v130/bozkurt21a/bozkurt21a.pdf
# but they aren't very "excitign"

In [ ]:
all_conv_inds = conv_beta_inds + [conv_index]
conv_betas = [all_run_dicts[ix]["beta"] for ix in all_conv_inds]
sortorder = np.argsort(conv_betas)
sorted_conv_inds = [all_conv_inds[s] for s in sortorder]
all_timewarpvae_indices =  [base_config_index] + beta_inds
timewarpvae_betas = [all_run_dicts[ix]["beta"] for ix in all_timewarpvae_indices]
twvae_sortorder = np.argsort(timewarpvae_betas)
sorted_twvae_inds = [all_timewarpvae_indices[s] for s in twvae_sortorder]

print("twvae:",sorted_twvae_inds[:2])
print("conv:",sorted_conv_inds)
# just plot the first 2 betas
labels = [f"TimewarpVAEbt{timewarpvae_betas[s]}" for s in twvae_sortorder[1:3]] + [f"Conv{conv_betas[s]}" for s in sortorder[:2]]+ ["DMP"]+ ["PCA"] + ["RIA"]
save_plots("BetaConvTwo",rateylim=500,
           toplot=sorted_twvae_inds[:2] + sorted_conv_inds[:2]+ [-2] + [-1]+ [ria_index], 
           labels=labels,
           colors=["C0"]*2 + ["C1"]*2 +["C3"]+["C2"]+["C4"],
          shapes=["o","X"]*2 + ["*"] + ["."] + ["^"],
          labeltextcolor="white",
          allow_partial=False)

In [ ]:
all_conv_inds = conv_beta_inds + [conv_index]
conv_betas = [all_run_dicts[ix]["beta"] for ix in all_conv_inds]
sortorder = np.argsort(conv_betas)
sorted_conv_inds = [all_conv_inds[s] for s in sortorder]
all_timewarpvae_indices =  [base_config_index] + beta_inds
timewarpvae_betas = [all_run_dicts[ix]["beta"] for ix in all_timewarpvae_indices]
twvae_sortorder = np.argsort(timewarpvae_betas)
sorted_twvae_inds = [all_timewarpvae_indices[s] for s in twvae_sortorder]

labels = [f"TimewarpVAEbt{timewarpvae_betas[s]}" for s in twvae_sortorder] + [f"Conv{conv_betas[s]}" for s in sortorder]
save_plots("BetaConv",rateylim=500,toplot=sorted_twvae_inds + sorted_conv_inds, 
           labels=labels,
           colors=["C0"]*4 + ["C1"]*4,
          shapes=["o","P","X","*"]*2,
          labeltextcolor="white")

In [ ]:
dtw_index
#trans_index
base_config_index

save_plots("TimewarpVAEDTW",rateylim=100,toplot=[base_config_index,dtw_index], 
           labels=["TimewarpVAE", "beta-VAE DTW---sp"],
           colors=["C0","C1"],
          shapes=["o","P"],
          labeltextcolor="white")